# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os


# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities = pd.read_csv('../cities.csv')
cities.head()


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Temp,Lat,Lng,Wind Speed
0,0,Albany,90,US,1585014867,86,33.01,42.60,-73.97,9.17
1,1,Puerto Ayora,93,EC,1585014868,82,79.00,-0.74,-90.35,7.00
2,2,Port-Gentil,100,GA,1585014869,82,80.96,-0.72,8.78,9.57
3,3,Mataura,100,NZ,1585014870,91,51.01,-46.19,168.86,11.01
4,4,Shubarshi,43,KZ,1585014871,72,33.08,48.59,57.19,13.76


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
cities = cities.dropna()
humidity = cities['Humidity'].astype(float)
locations = cities[['Lat', 'Lng']].astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(cities["Humidity"]),
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
pref_cities_df = cities.loc[(cities['Temp'] > 75) & (cities['Temp'] < 80) & (cities['Cloudiness'] < 5) & (cities['Wind Speed'] < 9), :]
pref_cities_df = pref_cities_df.dropna(how='any')
pref_cities_df.reset_index(inplace=True)
del pref_cities_df['index']
del pref_cities_df['Unnamed: 0']
pref_cities_df

,City,Cloudiness,Country,Date,Humidity,Temp,Lat,Lng,Wind Speed
0,Santiago del Estero,0,AR,1585014861,43,79.16,-27.80,-64.26,8.90
1,Srīvardhan,0,IN,1585014917,66,79.52,18.03,73.02,2.93
2,Ijuí,0,BR,1585014929,53,75.20,-28.39,-53.91,3.36
3,Barra de Tecoanapa,0,MX,1585015341,80,75.02,16.52,-98.75,0.25
4,Torit,3,SS,1585015375,58,78.44,4.41,32.57,2.04
5,Maïné Soroa,0,NE,1585015456,20,79.70,13.21,12.02,4.41


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotels = []
for c in range(len(pref_cities_df)):
    lat = pref_cities_df.loc[c]['Lat']
    lng = pref_cities_df.loc[c]['Lng']
    
    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types": 'hotel',
        "key": g_key
    }
    
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    request_json = requests.get(base_url, params=params).json()
    
    try:
        hotels.append(request_json['results'][0]['name'])
    except:
        hotels.append('')
pref_cities_df['Hotel Name'] = hotels
pref_cities_df = pref_cities_df.dropna(how='any')
pref_cities_df

,City,Cloudiness,Country,Date,Humidity,Temp,Lat,Lng,Wind Speed,Hotel Name
0,Santiago del Estero,0,AR,1585014861,43,79.16,-27.80,-64.26,8.90,Santiago del Estero
1,Srīvardhan,0,IN,1585014917,66,79.52,18.03,73.02,2.93,Shrivardhan
2,Ijuí,0,BR,1585014929,53,75.20,-28.39,-53.91,3.36,Ijuí
3,Barra de Tecoanapa,0,MX,1585015341,80,75.02,16.52,-98.75,0.25,Barra de Tecoanapa
4,Torit,3,SS,1585015375,58,78.44,4.41,32.57,2.04,Torit
5,Maïné Soroa,0,NE,1585015456,20,79.70,13.21,12.02,4.41,Maïné-Soroa


In [6]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotels = [info_box_template.format(**row) for index, row in pref_cities_df.iterrows()]
locations = pref_cities_df[["Lat", "Lng"]]
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0,0,150,0.4)',
    stroke_color='rgba(0,150,0,0.4)', 
    scale=2,
    info_box_content=hotels
)



In [7]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations)
fig.add_layer(hotel_layer)
fig

# Display Map

Figure(layout=FigureLayout(height='420px'))